In [153]:
import pandas as pd
import numpy as np
import chardet
import dask.dataframe as dd
import psutil
import os
import time


In [154]:
# Function to get memory usage in MB
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

file_path = 'globalterrorismdb_0718dist.csv'
with open(file_path, 'rb') as f:
    raw_data = f.read(10000)
    result = chardet.detect(raw_data)

# Load dataset using Pandas
start_time = time.time()
df = pd.read_csv(file_path, encoding=result['encoding'])
pandas_load_time = time.time() - start_time
pandas_memory_usage = get_memory_usage()

# Load dataset using Dask
start_time = time.time()
dask_df = dd.read_csv(file_path, encoding=result['encoding'])
dask_load_time = time.time() - start_time
dask_memory_usage = get_memory_usage()

print(f"Pandas Load Time: {pandas_load_time} seconds")
print(f"Pandas Memory Usage: {pandas_memory_usage} MB")
print(f"Dask Load Time: {dask_load_time} seconds")
print(f"Dask Memory Usage: {dask_memory_usage} MB")


C:\Users\ahmed\AppData\Local\Temp\ipykernel_20108\1648581182.py:13: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding=result['encoding'])


Pandas Load Time: 2.3142383098602295 seconds
Pandas Memory Usage: 594.4921875 MB
Dask Load Time: 0.03918337821960449 seconds
Dask Memory Usage: 597.640625 MB


In [155]:
pd.set_option('display.max_columns', None)
df.head(5)
df.columns
# df.shape


Index(['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended',
       'resolution', 'country', 'country_txt', 'region',
       ...
       'addnotes', 'scite1', 'scite2', 'scite3', 'dbsource', 'INT_LOG',
       'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related'],
      dtype='object', length=135)

In [156]:
# Choosing only the columns that are needed

df1 = df[
    ['iyear', 'imonth', 'iday', 'country_txt', 'extended',
     'success', 'suicide' ,'region_txt', 'city', 'latitude', 
     'longitude', 'attacktype1_txt', 'targtype1_txt', 'natlty1_txt',
     'gname','nkill', 'nwound', 'weaptype1_txt']
     ]


In [157]:
print(df1.columns, df1.shape)
df1.head(10)

Index(['iyear', 'imonth', 'iday', 'country_txt', 'extended', 'success',
       'suicide', 'region_txt', 'city', 'latitude', 'longitude',
       'attacktype1_txt', 'targtype1_txt', 'natlty1_txt', 'gname', 'nkill',
       'nwound', 'weaptype1_txt'],
      dtype='object') (181691, 18)


,iyear,imonth,iday,country_txt,extended,success,suicide,region_txt,city,latitude,longitude,attacktype1_txt,targtype1_txt,natlty1_txt,gname,nkill,nwound,weaptype1_txt
0,1970,7,2,Dominican Republic,0,1,0,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,Assassination,Private Citizens & Property,Dominican Republic,MANO-D,1.0,0.0,Unknown
1,1970,0,0,Mexico,0,1,0,North America,Mexico city,19.371887,-99.086624,Hostage Taking (Kidnapping),Government (Diplomatic),Belgium,23rd of September Communist League,0.0,0.0,Unknown
2,1970,1,0,Philippines,0,1,0,Southeast Asia,Unknown,15.478598,120.599741,Assassination,Journalists & Media,United States,Unknown,1.0,0.0,Unknown
3,1970,1,0,Greece,0,1,0,Western Europe,Athens,37.997490,23.762728,Bombing/Explosion,Government (Diplomatic),United States,Unknown,NaN,NaN,Explosives
4,1970,1,0,Japan,0,1,0,East Asia,Fukouka,33.580412,130.396361,Facility/Infrastructure Attack,Government (Diplomatic),United States,Unknown,NaN,NaN,Incendiary
5,1970,1,1,United States,0,1,0,North America,Cairo,37.005105,-89.176269,Armed Assault,Police,United States,Black Nationalists,0.0,0.0,Firearms
6,1970,1,2,Uruguay,0,0,0,South America,Montevideo,-34.891151,-56.187214,Assassination,Police,Uruguay,Tupamaros (Uruguay),0.0,0.0,Firearms
7,1970,1,2,United States,0,1,0,North America,Oakland,37.791927,-122.225906,Bombing/Explosion,Utilities,United States,Unknown,0.0,0.0,Explosives
8,1970,1,2,United States,0,1,0,North America,Madison,43.076592,-89.412488,Facility/Infrastructure Attack,Military,United States,New Year's Gang,0.0,0.0,Incendiary
9,1970,1,3,United States,0,1,0,North America,Madison,43.072950,-89.386694,Facility/Infrastructure Attack,Government (General),United States,New Year's Gang,0.0,0.0,Incendiary


In [158]:
(df1.isnull().sum()/len(df1))*100


iyear              0.000000
imonth             0.000000
iday               0.000000
country_txt        0.000000
extended           0.000000
success            0.000000
suicide            0.000000
region_txt         0.000000
city               0.239417
latitude           2.507554
longitude          2.508104
attacktype1_txt    0.000000
targtype1_txt      0.000000
natlty1_txt        0.858050
gname              0.000000
nkill              5.676120
nwound             8.977330
weaptype1_txt      0.000000
dtype: float64

In [159]:
df2 = df1.dropna()
(df2.isnull().sum()/len(df2))*100
df2.reset_index(drop=True, inplace=True)



In [160]:
df2.nunique()

iyear                 47
imonth                13
iday                  32
country_txt          202
extended               2
success                2
suicide                2
region_txt            12
city               32320
latitude           44295
longitude          44038
attacktype1_txt        9
targtype1_txt         22
natlty1_txt          212
gname               3291
nkill                179
nwound               238
weaptype1_txt         12
dtype: int64

In [161]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159029 entries, 0 to 159028
Data columns (total 18 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   iyear            159029 non-null  int64  
 1   imonth           159029 non-null  int64  
 2   iday             159029 non-null  int64  
 3   country_txt      159029 non-null  object 
 4   extended         159029 non-null  int64  
 5   success          159029 non-null  int64  
 6   suicide          159029 non-null  int64  
 7   region_txt       159029 non-null  object 
 8   city             159029 non-null  object 
 9   latitude         159029 non-null  float64
 10  longitude        159029 non-null  float64
 11  attacktype1_txt  159029 non-null  object 
 12  targtype1_txt    159029 non-null  object 
 13  natlty1_txt      159029 non-null  object 
 14  gname            159029 non-null  object 
 15  nkill            159029 non-null  float64
 16  nwound           159029 non-null  floa

In [162]:
df2 = df2.drop_duplicates()
df2.reset_index(drop=True, inplace=True)


In [163]:
for c in df2:
    print(df2[c].name ,df2[c].unique())

iyear [1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1986 1982
 1983 1984 1985 1987 1988 1989 1990 1991 1992 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017]
imonth [ 7  0  1  2  3  4  5  6  8  9 10 11 12]
iday [ 2  0  1  3  6  8  9 12 13 14 15 19 20 21 22 25 26 27 28 30 31  4  7 11
 16 17 18 23 24  5 10 29]
country_txt ['Dominican Republic' 'Mexico' 'Philippines' 'United States' 'Uruguay'
 'Italy' 'Guatemala' 'Venezuela' 'West Germany (FRG)' 'Switzerland'
 'Brazil' 'Egypt' 'Argentina' 'Lebanon' 'Japan' 'Jordan' 'Turkey'
 'Paraguay' 'East Germany (GDR)' 'United Kingdom' 'Greece' 'Nicaragua'
 'Belgium' 'Netherlands' 'Canada' 'Iran' 'Australia' 'Pakistan' 'Spain'
 'Ethiopia' 'Sweden' 'South Yemen' 'Cambodia' 'Israel' 'Poland' 'Panama'
 'West Bank and Gaza Strip' 'Ireland' 'India' 'Austria' 'France'
 'South Vietnam' 'Colombia' 'Brunei' 'Zaire'
 "People's Republic of the Congo" 'Portugal' 'Algeria' 'El 

In [164]:
columns_to_check = []

# Identify columns with more than 0.3% 'Unknown' values
for c in df2.columns:
    count_unknown = df2[c].astype(str).str.lower().eq('unknown').sum()
    if count_unknown > 0:
        print(f"Column '{c}' contains {count_unknown} 'Unknown' values out of {len(df2)}")
    if (count_unknown / len(df2)) * 100 < 30:
        columns_to_check.append(c)

# Drop rows with 'Unknown' values in identified columns
for c in columns_to_check:
    df2 = df2[~df2[c].astype(str).str.lower().eq('unknown')]

# Print the shape of the cleaned DataFrame to confirm the changes
print(f"Cleaned DataFrame shape: {df2.shape}")

df2.reset_index(drop=True, inplace=True)


Column 'city' contains 5380 'Unknown' values out of 147850
Column 'attacktype1_txt' contains 5409 'Unknown' values out of 147850
Column 'targtype1_txt' contains 3965 'Unknown' values out of 147850
Column 'gname' contains 71799 'Unknown' values out of 147850
Column 'weaptype1_txt' contains 10640 'Unknown' values out of 147850
Cleaned DataFrame shape: (128838, 18)


In [165]:
for c in df2:
    print(df2[c].name ,df2[c].unique())

iyear [1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1986 1982
 1983 1984 1985 1987 1988 1989 1990 1991 1992 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017]
imonth [ 1  2  3  4  5  6  7  8  9 10 11 12  0]
iday [ 1  2  3  6  8  9 12 13 14 15 19 21 22 25 26 27 28 30 31  4  7 11 16 17
 18 20 23 24  0  5 10 29]
country_txt ['United States' 'Uruguay' 'Italy' 'Philippines' 'West Germany (FRG)'
 'Switzerland' 'Guatemala' 'Egypt' 'Lebanon' 'Japan' 'Jordan' 'Turkey'
 'Paraguay' 'East Germany (GDR)' 'United Kingdom' 'Greece' 'Argentina'
 'Belgium' 'Netherlands' 'Canada' 'Australia' 'Pakistan' 'Spain'
 'Dominican Republic' 'Sweden' 'Iran' 'Venezuela' 'South Yemen' 'Cambodia'
 'Israel' 'Poland' 'Brazil' 'Panama' 'West Bank and Gaza Strip' 'Ireland'
 'India' 'Austria' 'France' 'South Vietnam' 'Colombia' 'Brunei' 'Zaire'
 "People's Republic of the Congo" 'Portugal' 'Algeria' 'El Salvador'
 'Mexico' 'Thailand' 'M

In [166]:
'''
to decide whether we're keeping or dropping the zero values in iday and imonth 
columns, Iwe need to check the som of the columns where day or month has zero
in respect with the approx date (we need to use the original df)

'''
# df_day_month_approx = df[(df['iday'] == 0) | (df['imonth'] == 0)][['iday', 'imonth', 'approxdate']]
# print(df_day_month_approx.head(10))

#count the number of zeroes in imonth and iday columns in the df2
zero_count_month = (df2['imonth'] == 0).sum()
print(f"Number of zeroes in 'imonth' column: {zero_count_month}")

zero_count_day = (df2['iday'] == 0).sum()
print(f"Number of zeroes in 'iday' column: {zero_count_day}")


'''
Decisions:
drop the iday column as we don't need the exact day of the attack
fill the zero values in imonth column with the mode of the column
'''

df2.drop('iday', axis=1, inplace=True)
df2['imonth'] = df2['imonth'].replace(0, df2['imonth'].mode()[0])
df2.reset_index(drop=True, inplace=True)


Number of zeroes in 'imonth' column: 6
Number of zeroes in 'iday' column: 385


In [167]:
for c in df2:
    print(df2[c].name ,df2[c].unique())

iyear [1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1986 1982
 1983 1984 1985 1987 1988 1989 1990 1991 1992 1994 1995 1996 1997 1998
 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012
 2013 2014 2015 2016 2017]
imonth [ 1  2  3  4  5  6  7  8  9 10 11 12]
country_txt ['United States' 'Uruguay' 'Italy' 'Philippines' 'West Germany (FRG)'
 'Switzerland' 'Guatemala' 'Egypt' 'Lebanon' 'Japan' 'Jordan' 'Turkey'
 'Paraguay' 'East Germany (GDR)' 'United Kingdom' 'Greece' 'Argentina'
 'Belgium' 'Netherlands' 'Canada' 'Australia' 'Pakistan' 'Spain'
 'Dominican Republic' 'Sweden' 'Iran' 'Venezuela' 'South Yemen' 'Cambodia'
 'Israel' 'Poland' 'Brazil' 'Panama' 'West Bank and Gaza Strip' 'Ireland'
 'India' 'Austria' 'France' 'South Vietnam' 'Colombia' 'Brunei' 'Zaire'
 "People's Republic of the Congo" 'Portugal' 'Algeria' 'El Salvador'
 'Mexico' 'Thailand' 'Morocco' 'Cyprus' 'Peru' 'Chile' 'Yugoslavia'
 'Ecuador' 'New Zealand' 'Zambia' 'Malaysia' 'Botswana' 'Kuwai

In [168]:
# # Define mapping dictionaries

# extended_map = {
#     0: 'Did Not Extend', 1: 'Extended'
# }

# success_map = {
#     0: 'Succeeded', 1: 'Failed'
# }

# suicide_map = {
#     0: 'Committed', 1: 'Did Not Commit'
# }


# df2['extended'] = df2['extended'].map(extended_map)
# df2['success'] = df2['success'].map(success_map)
# df2['suicide'] = df2['suicide'].map(suicide_map)

# # Display the DataFrame with mapped values
# df2.head(10)


In [169]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128838 entries, 0 to 128837
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   iyear            128838 non-null  int64  
 1   imonth           128838 non-null  int64  
 2   country_txt      128838 non-null  object 
 3   extended         128838 non-null  int64  
 4   success          128838 non-null  int64  
 5   suicide          128838 non-null  int64  
 6   region_txt       128838 non-null  object 
 7   city             128838 non-null  object 
 8   latitude         128838 non-null  float64
 9   longitude        128838 non-null  float64
 10  attacktype1_txt  128838 non-null  object 
 11  targtype1_txt    128838 non-null  object 
 12  natlty1_txt      128838 non-null  object 
 13  gname            128838 non-null  object 
 14  nkill            128838 non-null  float64
 15  nwound           128838 non-null  float64
 16  weaptype1_txt    128838 non-null  obje

In [170]:
# # Convert 'iyear' to string
# df2['iyear'] = df2['iyear'].astype(str)

# # Create a dictionary to map month numbers to month names
# month_map = {1: 'JAN', 2: 'FEB', 3: 'MAR', 4: 'APR', 5: 'MAY', 6: 'JUN',
#              7: 'JUL', 8: 'AUG', 9: 'SEP', 10: 'OCT', 11: 'NOV', 12: 'DEC'}

# # Map 'imonth' to month names
# df2['imonth'] = df2['imonth'].map(month_map)

In [171]:
# Convert 'latitude' and 'longitude' to string type

df2['latitude'] = df2['latitude'].astype(str)
df2['longitude'] = df2['longitude'].astype(str)


In [172]:
# Convert 'nkill' and 'nwound' to integer type
df2['nkill'] = df2['nkill'].fillna(0).astype('int64')
df2['nwound'] = df2['nwound'].fillna(0).astype('int64')

print(df2.dtypes)

iyear               int64
imonth              int64
country_txt        object
extended            int64
success             int64
suicide             int64
region_txt         object
city               object
latitude           object
longitude          object
attacktype1_txt    object
targtype1_txt      object
natlty1_txt        object
gname              object
nkill               int64
nwound              int64
weaptype1_txt      object
dtype: object


In [173]:
column_rename_map = {
    'country_txt': 'country_name',
    'region_txt': 'region',
    'attacktype1_txt': 'attack_type',
    'targtype1_txt': 'target_type',
    'natlty1_txt': 'nationality_of_target',
    'gname': 'group_name',
    'nkill': 'number_of_kills',
    'nwound': 'number_of_wounds',
    'weaptype1_txt': 'weapon_type'
}
df2 = df2.rename(columns=column_rename_map)
df2.columns

Index(['iyear', 'imonth', 'country_name', 'extended', 'success', 'suicide',
       'region', 'city', 'latitude', 'longitude', 'attack_type', 'target_type',
       'nationality_of_target', 'group_name', 'number_of_kills',
       'number_of_wounds', 'weapon_type'],
      dtype='object')

In [175]:
df2.shape

(128838, 17)

In [174]:
df2.to_csv('cleaned_globalterrorismdb_0718dist.csv', index=False)


In [176]:
df2.head(10)

,iyear,imonth,country_name,extended,success,suicide,region,city,latitude,longitude,attack_type,target_type,nationality_of_target,group_name,number_of_kills,number_of_wounds,weapon_type
0,1970,1,United States,0,1,0,North America,Cairo,37.005105,-89.176269,Armed Assault,Police,United States,Black Nationalists,0,0,Firearms
1,1970,1,Uruguay,0,0,0,South America,Montevideo,-34.891151,-56.187214,Assassination,Police,Uruguay,Tupamaros (Uruguay),0,0,Firearms
2,1970,1,United States,0,1,0,North America,Oakland,37.791927,-122.225906,Bombing/Explosion,Utilities,United States,Unknown,0,0,Explosives
3,1970,1,United States,0,1,0,North America,Madison,43.076592,-89.412488,Facility/Infrastructure Attack,Military,United States,New Year's Gang,0,0,Incendiary
4,1970,1,United States,0,1,0,North America,Madison,43.07295,-89.386694,Facility/Infrastructure Attack,Government (General),United States,New Year's Gang,0,0,Incendiary
5,1970,1,United States,0,0,0,North America,Baraboo,43.4685,-89.744299,Bombing/Explosion,Military,United States,"Weather Underground, Weathermen",0,0,Explosives
6,1970,1,United States,0,1,0,North America,Denver,39.758968,-104.876305,Facility/Infrastructure Attack,Military,United States,Left-Wing Militants,0,0,Incendiary
7,1970,1,Italy,0,1,0,Western Europe,Rome,41.890961,12.490069,Hijacking,Airports & Aircraft,United States,Unknown,0,0,Firearms
8,1970,1,United States,0,1,0,North America,Detroit,42.331685,-83.047924,Facility/Infrastructure Attack,Government (General),United States,Left-Wing Militants,0,0,Incendiary
9,1970,1,United States,0,1,0,North America,Rio Piedras,18.386932,-66.061127,Facility/Infrastructure Attack,Business,United States,Armed Commandos of Liberation,0,0,Incendiary
